<a href="https://colab.research.google.com/github/YosyValero/jvalero59_TFG/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline TFG

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [4]:
path_tsv = '/content/drive/MyDrive/Dataset/tsv/'

In [5]:
df_dissea =  pd.read_csv(path_tsv + 'dissea.tsv', sep='\t', header=0).loc[:, ('tweets_id', 'type')].rename(columns={'tweets_id': 'tweet_id'})

In [6]:
df_prof =  pd.read_csv(path_tsv + 'prof.tsv', sep='\t', header=0).loc[:, ('tweet_id', 'type')]

In [7]:
df_prof.drop(df_prof[df_prof['type']=='-'].index, inplace=True)

In [8]:
df_prof = df_prof.replace(['PROFESION', 'SITUACION_LABORAL', 'ACTIVIDAD', 'FIGURATIVA'], ['NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD'])

In [9]:
df_prof_norep = df_prof.drop_duplicates(ignore_index=True)

In [10]:
df_disse_norep = df_dissea.drop_duplicates(ignore_index=True)

In [11]:
muestras_min = min([len(df_disse_norep), len(df_prof_norep)])

In [12]:
muestras = 50

In [13]:
if muestras_min < muestras:
  muestras = muestras_min

In [14]:
df_xamples_diss = df_disse_norep.head(muestras)

In [15]:
df_xamples_nodiss = df_prof_norep.head(muestras)

In [16]:
df_xamples_nodiss = df_xamples_nodiss.rename(index=lambda x: x + muestras)

In [17]:
df_tot = pd.concat([df_xamples_diss, df_xamples_nodiss])


In [ ]:
df_tot

In [19]:
df_tot.insert(2, 'content_tuit', value='')

In [20]:
def get_tuit(type_tuit, name_tuit):
  sub_path = path_tsv + 'train-valid-txt-files/'
  if type_tuit == 'ENFERMEDAD':
    path_tuit = sub_path + 'dissea'
  else:
    path_tuit = sub_path + 'prof'

  path_tuit = path_tuit + '/training/' + name_tuit + '.txt'
  tuit = open(path_tuit, mode='r', encoding='utf-8')
  content_tuit = tuit.read()
  tuit.close()

  return content_tuit

In [21]:
for i in range(len(df_tot)):
    name_tuit = df_tot.loc[i, 'tweet_id']
    type_tuit = df_tot.loc[i, 'type']
    df_tot.at[i, 'content_tuit'] = get_tuit(type_tuit, str(name_tuit))

In [ ]:
df_tot

In [22]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [23]:
symbols = [
    '¿','?','~','`','!','¡','@','$','%','^',
    '*','(',')','_','-','+','=','{','}','[',
    ']','\\',';','<','>','/', '.', ',','&','\n'
    ,'\r','\t', '|', '“', '"', '–', '”', '©', '-', '—',
    '…', ';', ':', '‘','’', '...', '#', '«', '»', '``',
    "''", '..', '...', '--'
]

In [24]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [25]:
def limpia_tuit_sp(text):
    str_limp = ""
    stp_wors_mas_sym = stopwords.words('spanish') + symbols

    # Eliminamos las urls de los tuits
    text = re.sub('http[s]*://[a-zA-Z0-9/.-]*|www.[a-zA-Z0-9/.-]*', '', text)

    # Eliminamos numeros
    text = re.sub('[0-9]*','', text)
    # print(text)

    # Eliminamos emoticonos
    text = remove_emoji(text)

    # Eliminamos otros caracteres
    text = re.sub('[¿,_,?,¡,!,-]*','', text)

    # Tokenizamos los tuits
    text_tok = word_tokenize(text)
    for t_tok in text_tok:
        if not t_tok in stp_wors_mas_sym:
            str_limp = str_limp + " " + t_tok
    str_limp =  str_limp.lower()
    return str_limp

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

### A partir del dataset convertimos los tuits y sus type (ENFERMEDAD, NO_ENFERMEDAD) en 2 arrays

In [116]:
x, y = np.asarray(df_tot['content_tuit']), np.asarray(df_tot['type'])


In [117]:
# Troceamos el dataset en crudo con vines didácticos
x_train_raw, x_test_raw, y_train_raw, y_test_raw = train_test_split(x, y, train_size=70, random_state=42)

Limpiamos los tuit

In [74]:
tuits_limpios = [limpia_tuit_sp(t) for t in x]

### Conversión de los datos textuales a númericos, usos:
*   CountVectorizer
*   Tfidf

In [75]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [136]:
count_vectorizer = CountVectorizer()
X_numer_count = count_vectorizer.fit_transform(tuits_limpios).toarray()

In [77]:
tfid_vectorizer = TfidfVectorizer()
X_numer_tfid = tfid_vectorizer.fit_transform(tuits_limpios).toarray()

### Importamos las librerias para implementar el modelo mediante Support Vector Mahine y la división del dataset es subset entrenamiento (train) y testeo (test)

In [137]:
from sklearn import svm
from sklearn.model_selection import train_test_split
clf = svm.SVC(kernel='linear')

SVM_CountVectorized

In [138]:
x_train, x_test, y_train, y_test = train_test_split(X_numer_count, y, train_size=70, random_state=42)
clf.fit(x_train, y_train)

SVC(kernel='linear')

Se va a probar la prección del modelo definido utilizado algunos tuits del subset test

In [139]:
tuits_pruebas_limpios = [limpia_tuit_sp(t) for t in x_test_raw]

In [140]:
for x_tuit, y_tuit in zip(tuits_pruebas_limpios, y_test_raw):
  print(x_tuit, y_tuit)


 libertad expresión atacar jueces libertad expresión atacar rey mitad pandemia mientras reclaman unidad pero vox libertad expresion criticar inigualable gestión sanitaria gobierno no límite NO_ENFERMEDAD
 teresa ribera ministra podemita transición ecológica tocarse huevos pregunta porqué portugal menos infectados fallecidos españa responde virus viene este oeste en manos cobra .€ mes NO_ENFERMEDAD
 foodtrucks ofrecen caldo caliente bebidas bocadillos transportistas mercamadridhoy vía madridiario solidaridad yomequedoencasa foodtruck NO_ENFERMEDAD
 me pesa publicar quisiera trabajar les mando abrazo dejo canal you tube muchas charlas informativas tdah autismo nurodiversidad trastorno estres postraumático depresión ansiedad ENFERMEDAD
 yo asperger esquizofrenia según respuesta puede tener ambos diagnósticos entonces puedo tener síndrome asperger si esquizofrenia ENFERMEDAD
 ayer tomé super combo antihistamínicos pseudoefedrina hoy acabo desayunar enanthyum fuertota gripe vacuna reinas im

In [141]:
docs_news = ['Sin él, mi agorafobia era soportable, al igual que mi ansiedad: el tabaco potencia nuestras taras físicas y psicológicas. No me veía yo por el Amazonas en una lancha con mi cámara y con una crisis de pánico, previa crisis de ansiedad, previa hiperventilación.',\
            '"Aspiramos a individualizar el tratamiento en trombosis y cáncer y para esto tenemos que buscar biomarcadores"  Dra Vanesa Pachón #cáncer #revisionesencáncer #RevCáncer2019 #oncología']
docs_news_limpios = [limpia_tuit_sp(t) for t in docs_news]

In [142]:
docs_news_count = count_vectorizer.transform(docs_news_limpios).toarray()

In [143]:
predicted = clf.predict(docs_news_count)
for doc, categoria in zip(docs_news, predicted):
    print('%r => %s' % (doc, categoria))
print()

'Sin él, mi agorafobia era soportable, al igual que mi ansiedad: el tabaco potencia nuestras taras físicas y psicológicas. No me veía yo por el Amazonas en una lancha con mi cámara y con una crisis de pánico, previa crisis de ansiedad, previa hiperventilación.' => ENFERMEDAD
'"Aspiramos a individualizar el tratamiento en trombosis y cáncer y para esto tenemos que buscar biomarcadores"  Dra Vanesa Pachón #cáncer #revisionesencáncer #RevCáncer2019 #oncología' => ENFERMEDAD



In [144]:
tuits_pruebas_count = count_vectorizer.transform(tuits_pruebas_limpios).toarray()

In [146]:
predicted = clf.predict(tuits_pruebas_count)
for doc, categoria in zip(tuits_pruebas_limpios, predicted):
    print('%r => %s' % (doc, categoria))
print()

' libertad expresión atacar jueces libertad expresión atacar rey mitad pandemia mientras reclaman unidad pero vox libertad expresion criticar inigualable gestión sanitaria gobierno no límite' => NO_ENFERMEDAD
' teresa ribera ministra podemita transición ecológica tocarse huevos pregunta porqué portugal menos infectados fallecidos españa responde virus viene este oeste en manos cobra .€ mes' => NO_ENFERMEDAD
' foodtrucks ofrecen caldo caliente bebidas bocadillos transportistas mercamadridhoy vía madridiario solidaridad yomequedoencasa foodtruck' => NO_ENFERMEDAD
' me pesa publicar quisiera trabajar les mando abrazo dejo canal you tube muchas charlas informativas tdah autismo nurodiversidad trastorno estres postraumático depresión ansiedad' => ENFERMEDAD
' yo asperger esquizofrenia según respuesta puede tener ambos diagnósticos entonces puedo tener síndrome asperger si esquizofrenia' => NO_ENFERMEDAD
' ayer tomé super combo antihistamínicos pseudoefedrina hoy acabo desayunar enanthyum fu

In [112]:
clf.score(x_test, y_test)

0.7666666666666667

SVM_TfidfVectorizer

In [101]:
x_train, x_test, y_train, y_test = train_test_split(X_numer_tfid, y, train_size=70, random_state=42)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5333333333333333

In [ ]:
clf.set_params(kernel='rbf').fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)